In [1]:
import re
import sys
sys.path.append("..")
import lib

In [ ]:
MONKEYS = []

class Monkey():
    def __init__(self):
        self.test = NotImplemented
        self.worry_levels = [NotImplemented]
        self.action_true = NotImplemented
        self.action_false = NotImplemented
        pass

    def __repr__(self):
        pass

In [9]:
def parse_monkey(input : str):
    pass

def parse_split_into_monkeys(input : str):
    regex = re.split("\n{2}", input)
    MONKEYS = [parse_monkey(regex[idx]) for idx in len(regex)]
    return regex

In [10]:
input = lib.read_file_as_one("test_input.txt")
parse_split_into_monkeys(input)

['Monkey 0:\n  Starting items: 79, 98\n  Operation: new = old * 19\n  Test: divisible by 23\n    If true: throw to monkey 2\n    If false: throw to monkey 3',
 'Monkey 1:\n  Starting items: 54, 65, 75, 74\n  Operation: new = old + 6\n  Test: divisible by 19\n    If true: throw to monkey 2\n    If false: throw to monkey 0',
 'Monkey 2:\n  Starting items: 79, 60, 97\n  Operation: new = old * old\n  Test: divisible by 13\n    If true: throw to monkey 1\n    If false: throw to monkey 3',
 'Monkey 3:\n  Starting items: 74\n  Operation: new = old + 3\n  Test: divisible by 17\n    If true: throw to monkey 0\n    If false: throw to monkey 1']